In [126]:
import pandas as pd
data = pd.read_csv("data.csv")

/home/cm/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (44) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [127]:
data.head()

,Unnamed: 0,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,...,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment
0,109,SL,2015-11-01,85.2,-1.8461,6.1603,Kelvin Herrera,518595.0,516969.0,field_out,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,Standard,Standard
1,112,FF,2015-11-01,97.7,-1.6600,6.1500,Kelvin Herrera,446263.0,516969.0,field_out,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,Infield shift,Strategic
2,117,FT,2015-11-01,96.2,-1.9189,6.1301,Kelvin Herrera,501571.0,516969.0,field_out,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,Standard,Standard
3,120,SI,2015-11-01,97.1,-1.1903,6.2683,Jeurys Familia,460086.0,544727.0,field_out,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,Standard,Strategic
4,127,SI,2015-11-01,96.7,-1.2267,6.3665,Jeurys Familia,521692.0,544727.0,field_out,...,2.0,1.0,1.0,2.0,1.0,2.0,1.0,2.0,Strategic,Standard


In [128]:
data = data.loc[data['inning'] <= 9]

In [129]:
data.head()

,Unnamed: 0,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,...,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment
0,109,SL,2015-11-01,85.2,-1.8461,6.1603,Kelvin Herrera,518595.0,516969.0,field_out,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,Standard,Standard
1,112,FF,2015-11-01,97.7,-1.6600,6.1500,Kelvin Herrera,446263.0,516969.0,field_out,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,Infield shift,Strategic
2,117,FT,2015-11-01,96.2,-1.9189,6.1301,Kelvin Herrera,501571.0,516969.0,field_out,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,Standard,Standard
3,120,SI,2015-11-01,97.1,-1.1903,6.2683,Jeurys Familia,460086.0,544727.0,field_out,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,Standard,Strategic
4,127,SI,2015-11-01,96.7,-1.2267,6.3665,Jeurys Familia,521692.0,544727.0,field_out,...,2.0,1.0,1.0,2.0,1.0,2.0,1.0,2.0,Strategic,Standard


In [130]:
data = data.drop(axis=1, columns=['Unnamed: 0'])

In [131]:
data.head()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment
0,SL,2015-11-01,85.2,-1.8461,6.1603,Kelvin Herrera,518595.0,516969.0,field_out,hit_into_play,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,Standard,Standard
1,FF,2015-11-01,97.7,-1.6600,6.1500,Kelvin Herrera,446263.0,516969.0,field_out,hit_into_play,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,Infield shift,Strategic
2,FT,2015-11-01,96.2,-1.9189,6.1301,Kelvin Herrera,501571.0,516969.0,field_out,hit_into_play,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,Standard,Standard
3,SI,2015-11-01,97.1,-1.1903,6.2683,Jeurys Familia,460086.0,544727.0,field_out,hit_into_play,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,Standard,Strategic
4,SI,2015-11-01,96.7,-1.2267,6.3665,Jeurys Familia,521692.0,544727.0,field_out,hit_into_play_score,...,2.0,1.0,1.0,2.0,1.0,2.0,1.0,2.0,Strategic,Standard


In [132]:
data['events'].unique()

array(['field_out', 'double', 'strikeout', 'grounded_into_double_play',
       'single', 'field_error', 'force_out', 'home_run', 'double_play',
       'strikeout_double_play', 'fielders_choice', 'triple',
       'fielders_choice_out', 'catcher_interf', 'other_out',
       'triple_play'], dtype=object)

In [133]:
events = ['field_out', 'double', 'strikeout',
       'grounded_into_double_play', 'single', 'field_error',
       'force_out', 'home_run', 'double_play', 'strikeout_double_play',
       'fielders_choice', 'triple', 'fielders_choice_out', 'catcher_interf', 'other_out', 'triple_play']

#data = data[data['events'].isin(events)]

In [134]:
batter = data.groupby(['batter', 'game_year', 'inning']).count()
batter = batter[['pitch_type']]
# batter["AB"] = batter[['ba].sum()

In [135]:
# def set_ab(row):
#     print(row)
#     row['AB'] = row['pitch_type'].sum()
# batter_with_ab = batter.apply(set_ab, axis=1)

In [136]:
#batter_with_ab

In [137]:
batter = batter.reset_index()

In [138]:
batter = batter.groupby(['batter', 'game_year']).agg({'pitch_type':'sum'})
batter = batter.rename(index=str, columns={"pitch_type": "AB"})

In [139]:
batter.head()

AB
batter   game_year    
112526.0 2015.0     58
         2016.0     60
         2017.0     19
         2018.0      4
115629.0 2015.0      1

In [140]:
hits_data = data[data['events'].isin(['single', 'double', 'triple', 'home_run'])]
hits_data.head()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment
6,FF,2015-11-01,94.8,-1.3516,6.1937,Matt Harvey,543333.0,518774.0,double,hit_into_play_score,...,2.0,0.0,0.0,2.0,0.0,2.0,0.0,2.0,Standard,Strategic
15,FF,2015-11-01,97.2,-2.2647,5.9816,Kelvin Herrera,624424.0,516969.0,single,hit_into_play_no_out,...,2.0,0.0,2.0,0.0,0.0,2.0,2.0,0.0,Standard,Standard
19,FT,2015-11-01,96.3,-1.2401,6.2293,Matt Harvey,519058.0,518774.0,single,hit_into_play_no_out,...,2.0,0.0,0.0,2.0,0.0,2.0,0.0,2.0,Strategic,Standard
23,SI,2015-11-01,94.6,-1.4083,5.2756,Edinson Volquez,431151.0,450172.0,single,hit_into_play_no_out,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,Standard,Standard
26,SL,2015-11-01,91.8,-1.3545,5.9907,Matt Harvey,450314.0,518774.0,single,hit_into_play_no_out,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,Standard,Strategic


In [141]:
batter_data = hits_data.groupby(['batter', 'game_year', 'inning']).count()
batter_data = batter_data[['pitch_type']]
batter_data = batter_data.rename(index=str, columns={"pitch_type": "H"})
batter_data = batter_data.reset_index()
batter_data['inning'] = batter_data['inning'].astype('Float64')
batter_data['WH'] = batter_data['inning']*batter_data['H']
batter_data.head()

,batter,game_year,inning,H,WH
0,112526.0,2015.0,2.0,1,2.0
1,112526.0,2015.0,3.0,1,3.0
2,112526.0,2015.0,4.0,2,8.0
3,112526.0,2015.0,5.0,3,15.0
4,112526.0,2015.0,7.0,1,7.0


In [142]:
batter_iaba = batter_data.groupby(['batter', 'game_year']).agg({'WH':'sum'})
batter_iaba.head()

WH
batter   game_year       
112526.0 2015.0      35.0
         2016.0      21.0
116338.0 2015.0     568.0
120074.0 2015.0     602.0
         2016.0     774.0

In [143]:
batter = batter.loc[batter['AB'] >= 200]

In [144]:
batter_iaba = pd.merge(batter_iaba, batter, on=['batter','game_year'], how='inner')

In [145]:
batter_iaba['IABA'] = batter_iaba['WH']/batter_iaba['AB']
batter_iaba.head()

WH   AB      IABA
batter   game_year                      
116338.0 2015.0     568.0  518  1.096525
120074.0 2015.0     602.0  519  1.159923
         2016.0     774.0  536  1.444030
121347.0 2015.0     602.0  518  1.162162
         2016.0     198.0  224  0.883929

In [146]:
data.columns

Index(['pitch_type', 'game_date', 'release_speed', 'release_pos_x',
       'release_pos_z', 'player_name', 'batter', 'pitcher', 'events',
       'description', 'spin_dir', 'spin_rate_deprecated',
       'break_angle_deprecated', 'break_length_deprecated', 'zone', 'des',
       'game_type', 'stand', 'p_throws', 'home_team', 'away_team', 'type',
       'hit_location', 'bb_type', 'balls', 'strikes', 'game_year', 'pfx_x',
       'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b',
       'outs_when_up', 'inning', 'inning_topbot', 'hc_x', 'hc_y',
       'tfs_deprecated', 'tfs_zulu_deprecated', 'fielder_2', 'umpire', 'sv_id',
       'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot',
       'hit_distance_sc', 'launch_speed', 'launch_angle', 'effective_speed',
       'release_spin_rate', 'release_extension', 'game_pk', 'pitcher.1',
       'fielder_2.1', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6',
       'fielder_7', 'fielder_8', 'fielder_9', 'release_pos_y',
       'estima

In [147]:
batter_team = data[['batter', 'inning_topbot', 'home_team', 'away_team', 'game_year']]
home_team = batter_team[batter_team['inning_topbot'] == "Bot"]
home_team["team"] = home_team['home_team']
away_team = batter_team[batter_team['inning_topbot'] == "Top"]
away_team["team"] = away_team['away_team']

/home/cm/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/cm/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [148]:
batter_team = pd.concat([home_team, away_team])

In [149]:
batter_team.head()

,batter,inning_topbot,home_team,away_team,game_year,team
0,518595.0,Bot,NYM,KC,2015.0,NYM
1,446263.0,Bot,NYM,KC,2015.0,NYM
2,501571.0,Bot,NYM,KC,2015.0,NYM
7,502517.0,Bot,NYM,KC,2015.0,NYM
8,431151.0,Bot,NYM,KC,2015.0,NYM


In [150]:
batter_iaba_with_team = batter_iaba.reset_index()
batter_iaba_with_team['game_year'] = batter_iaba_with_team['game_year'].astype('Float64')
batter_iaba_with_team['batter'] = batter_iaba_with_team['batter'].astype('Float64')
batter_iaba_with_team = pd.merge(batter_iaba_with_team, batter_team[['batter', 'game_year', 'team']], on=['batter', 'game_year'], how='inner')

In [151]:
# batter_iaba_with_team = batter_iaba_with_team.groupby(['batter', 'game_year'])
batter_iaba_with_team.head()

,batter,game_year,WH,AB,IABA,team
0,116338.0,2015.0,568.0,518,1.096525,MIN
1,116338.0,2015.0,568.0,518,1.096525,MIN
2,116338.0,2015.0,568.0,518,1.096525,MIN
3,116338.0,2015.0,568.0,518,1.096525,MIN
4,116338.0,2015.0,568.0,518,1.096525,MIN


In [152]:
#df = batter_iaba_with_team.to_csv("iaba_with_team.csv")

In [153]:
teams = pd.read_csv('Teams.csv')

In [154]:
teams = teams.loc[teams['yearID'] >= 2015]
teams = teams[['yearID', 'teamID',  'G', 'W', 'L', 'R', 'AB', 'H', 'BB', 'HBP']]
teams['BA'] = teams['H']/teams['AB']
teams['OBP'] = (teams['H']+teams['BB']+teams['HBP'])/(teams['AB']+teams['BB']+teams['HBP'])
teams.head()

,yearID,teamID,G,W,L,R,AB,H,BB,HBP,BA,OBP
2775,2015,ARI,162,79,83,720,5649,1494,490.0,33.0,0.264472,0.326798
2776,2015,ATL,162,67,95,573,5420,1361,471.0,44.0,0.251107,0.316091
2777,2015,BAL,162,81,81,713,5485,1370,418.0,51.0,0.249772,0.308868
2778,2015,BOS,162,78,84,748,5640,1495,478.0,46.0,0.265071,0.327547
2779,2015,CHA,162,76,86,622,5533,1381,404.0,65.0,0.249593,0.308231


In [155]:
# df = teams.to_csv("teams15-18.csv")

In [156]:
lahman_batter = pd.read_csv('Batting.csv')
lahman_batter = lahman_batter.loc[(lahman_batter['yearID'] >= 2015) & (lahman_batter['AB'] >= 200)]

In [157]:
lahman_batter = lahman_batter[['playerID', 'yearID', 'teamID', 'G', 'AB', 'R', 'H', 'RBI', 'BB', 'HBP']]
lahman_batter['BA'] = lahman_batter['H']/lahman_batter['AB']
lahman_batter['OBP'] = (lahman_batter['H']+lahman_batter['BB']+lahman_batter['HBP'])/(lahman_batter['AB']+lahman_batter['BB']+lahman_batter['HBP'])
lahman_batter = lahman_batter.reset_index()
lahman_batter = lahman_batter.drop(axis=1, columns=['index'])
lahman_batter.head()

,playerID,yearID,teamID,G,AB,R,H,RBI,BB,HBP,BA,OBP
0,abreujo02,2015,CHA,154,613,88,178,101.0,39,15.0,0.290375,0.347826
1,ahmedni01,2015,ARI,134,421,49,95,34.0,29,1.0,0.225653,0.277162
2,alonsyo01,2015,SDN,103,354,50,100,31.0,42,3.0,0.282486,0.363409
3,altuvjo01,2015,HOU,154,638,86,200,66.0,33,9.0,0.313480,0.355882
4,alvarpe01,2015,PIT,150,437,60,106,77.0,48,2.0,0.242563,0.320329


In [158]:
# df = lahman_batter.to_csv("lahman_batter.csv")

In [159]:
ids = playerid_reverse_lookup(lahman_batter['playerID'], "bbref")[['key_mlbam', 'key_bbref']]

Gathering player lookup table. This may take a moment.


In [160]:
ids = ids.rename(index=str, columns={"key_bbref": "playerID", "key_mlbam":"batter"})
ids.head()

,batter,playerID
0,547989,abreujo02
1,660670,acunaro01
2,542436,adamecr01
3,642715,adamewi01
4,571431,adamsma01


In [161]:
unique = batter_iaba_with_team.drop_duplicates().reset_index()
unique = unique.drop(axis=1, columns=['index'])
unique = unique.rename(index=str, columns={"game_year": "yearID"})
unique.head()

,batter,yearID,WH,AB,IABA,team
0,116338.0,2015.0,568.0,518,1.096525,MIN
1,120074.0,2015.0,602.0,519,1.159923,BOS
2,120074.0,2016.0,774.0,536,1.444030,BOS
3,121347.0,2015.0,602.0,518,1.162162,NYY
4,121347.0,2016.0,198.0,224,0.883929,NYY


In [162]:
b = pd.merge(lahman_batter, ids, on='playerID', how='inner')
b.head()

,playerID,yearID,teamID,G,AB,R,H,RBI,BB,HBP,BA,OBP,batter
0,abreujo02,2015,CHA,154,613,88,178,101.0,39,15.0,0.290375,0.347826,547989
1,abreujo02,2016,CHA,159,624,67,183,100.0,47,15.0,0.293269,0.357143,547989
2,abreujo02,2017,CHA,156,621,95,189,102.0,35,15.0,0.304348,0.356185,547989
3,abreujo02,2018,CHA,128,499,68,132,78.0,37,11.0,0.264529,0.329068,547989
4,ahmedni01,2015,ARI,134,421,49,95,34.0,29,1.0,0.225653,0.277162,605113


In [163]:
b = pd.merge(b, unique, on=['batter', 'yearID'], how='inner')
b = b.rename(index=str, columns={"AB_x": "AB", "AB_y": "AB_1-9"})
b.head()

,playerID,yearID,teamID,G,AB,R,H,RBI,BB,HBP,BA,OBP,batter,WH,AB_1-9,IABA,team
0,abreujo02,2015,CHA,154,613,88,178,101.0,39,15.0,0.290375,0.347826,547989,773.0,602,1.284053,CWS
1,abreujo02,2016,CHA,159,624,67,183,100.0,47,15.0,0.293269,0.357143,547989,788.0,610,1.291803,CWS
2,abreujo02,2017,CHA,156,621,95,189,102.0,35,15.0,0.304348,0.356185,547989,837.0,613,1.365416,CWS
3,abreujo02,2018,CHA,128,499,68,132,78.0,37,11.0,0.264529,0.329068,547989,631.0,490,1.287755,CWS
4,ahmedni01,2015,ARI,134,421,49,95,34.0,29,1.0,0.225653,0.277162,605113,443.0,410,1.080488,ARI


In [164]:
b["OBP"].corr(b["IABA"], method='pearson')

0.501454799296949